# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
from pathlib import Path

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,maceio,-9.6658,-35.7353,28.69,74,40,6.17,BR,1727527111
1,1,yigo village,13.5361,144.8886,27.62,87,75,2.06,GU,1727527113
2,2,revda,56.8010,59.9303,13.16,50,100,5.43,RU,1727527115
3,3,grytviken,-54.2811,-36.5092,2.10,92,100,6.40,GS,1727527117
4,4,bethel,41.3712,-73.4140,18.62,85,100,1.54,US,1727527119


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo = True,
    tiles = "OSM",
    color= "City",
    alpha=0.6,        
    hover_cols=['City', 'Humidity']
)
# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) &
    (city_data_df['Max Temp'] <= 27) &
    (city_data_df['Cloudiness'] == 0) &
    (city_data_df['Wind Speed'] <= 4.5)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display the filtered DataFrame
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
102,102,fort bragg,35.1390,-79.0060,21.91,91,0,3.09,US,1727527322
150,150,cascais,38.6979,-9.4215,22.53,54,0,2.57,PT,1727527420
167,167,mahina,-17.5065,-149.4890,24.87,83,0,1.03,PF,1727527457
183,183,east london,-33.0153,27.9116,22.52,60,0,2.57,ZA,1727527489
193,193,buka,40.8108,69.1986,21.24,22,0,3.47,UZ,1727527509
232,232,bernardo de irigoyen,-26.2552,-53.6458,23.11,38,0,1.78,AR,1727527596
302,302,gotsucho,35.0000,132.2167,21.07,85,0,4.41,JP,1727527742
333,333,andilamena,-17.0167,48.5833,25.71,35,0,4.06,MG,1727527806
379,379,turi,40.9173,17.0212,26.42,25,0,2.76,IT,1727527902
382,382,haiku-pauwela,20.9219,-156.3051,22.36,80,0,3.60,US,1727527908


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
102,fort bragg,US,35.1390,-79.0060,91,
150,cascais,PT,38.6979,-9.4215,54,
167,mahina,PF,-17.5065,-149.4890,83,
183,east london,ZA,-33.0153,27.9116,60,
193,buka,UZ,40.8108,69.1986,22,
232,bernardo de irigoyen,AR,-26.2552,-53.6458,38,
302,gotsucho,JP,35.0000,132.2167,85,
333,andilamena,MG,-17.0167,48.5833,35,
379,turi,IT,40.9173,17.0212,25,
382,haiku-pauwela,US,20.9219,-156.3051,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:

# Set parameters to search for a hotel
radius = 10000
params = {  
    'categories': 'accommodation.hotel',
    'limit': 1,  # Get the first result
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
cascais - nearest hotel: Cascais Hotel
mahina - nearest hotel: Motu Martin
east london - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
bernardo de irigoyen - nearest hotel: Irigoyen
gotsucho - nearest hotel: No hotel found
andilamena - nearest hotel: Hotely Magic
turi - nearest hotel: Hotel Esperia
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
horizontina - nearest hotel: Ouro Verde
archangelos - nearest hotel: Semina
pahrump - nearest hotel: Holiday Inn Express & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
102,fort bragg,US,35.1390,-79.0060,91,Airborne Inn Lodging
150,cascais,PT,38.6979,-9.4215,54,Cascais Hotel
167,mahina,PF,-17.5065,-149.4890,83,Motu Martin
183,east london,ZA,-33.0153,27.9116,60,No hotel found
193,buka,UZ,40.8108,69.1986,22,No hotel found
232,bernardo de irigoyen,AR,-26.2552,-53.6458,38,Irigoyen
302,gotsucho,JP,35.0000,132.2167,85,No hotel found
333,andilamena,MG,-17.0167,48.5833,35,Hotely Magic
379,turi,IT,40.9173,17.0212,25,Hotel Esperia
382,haiku-pauwela,US,20.9219,-156.3051,80,Inn At Mama's Fish House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo = True,
    tiles = "OSM",
    color= "City",
    alpha=0.6,        
    hover_cols=['City','Humidity','Hotel Name','Country'] 
)
# Display the map plot_1
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)